In [1]:
import numpy as np

import os
def build(fig, name):
    with open(f'temp/{name}.tex', 'w') as f:
        f.write(fig)
    x = os.system(f"latexmk -pdf temp/{name}.tex")
    print(x)
    
class PDF(object):
    def __init__(self, pdf, size=(200,200)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

In [2]:
import tikzinpy
from tikzinpy.shapes import *
from tikzinpy.shapes.datashapes import *

In [3]:
# This is just some data to test with
data_x = np.linspace(-2, 6, 100)
data_e = np.random.uniform(0,1, size = 100)
data_y = 0.5*data_x + data_e

# Low level components
The first example shows how to build a figure from atomics. All of the higher level functionality uses these atomics to simplify building figures

In [4]:
base = tikzinpy.tikzBase()
print(base.figure)


        \documentclass[tikz, border=2mm]{standalone}
        \usepackage{tikz}
        \begin{document}    
            \begin{tikzpicture}
        
            \end{tikzpicture}
        \end{document}
        


In [5]:
# This simply generates the points we want to use for 
# the axis ticks
points = tikzinpy.number_range(-2,6, decimals = 0, remove_zero = True)
points

'-2,-1,1,2,3,4,5,6'

In [ ]:
# This adds arrows going along the X and y axis of the plot
base += tikzinpy.arrow(-2, 6,0,0, 'x', align='right', width='thick')
base += tikzinpy.arrow(0, 0,-2,6, 'y', align='above', width='thick')

print(base.figure)

In [ ]:
# This add axis ticks along the horizontal and vertical axis
# one tick for each point in points
base += tikzinpy.ticks(points, 'h')
base += tikzinpy.ticks(points, 'v')
print(base.figure, 'atomic')

In [ ]:
# This add tick marks on top of the tick lines
base += tikzinpy.tickmarks(points, 'h', offset = '-5pt', fontsize = 'small')
base += tikzinpy.tickmarks(points, 'v', offset = '-5pt', fontsize = 'small')

print(base.figure)

In [ ]:
# Now to plot a bunch of points on the graph
for x,y in zip(data_x, data_y):
    base += point(x,y, shape = 'hollow', color = 'blue')

In [ ]:
build(base.figure, 'atomic')
PDF('atomic.pdf', size=(700,450))

In [ ]:
# The figure is essentially stored as a dict.
base._body

In [ ]:
# You can remove an element from the figure by doing
base -= 'arrow1'

# High level components
Now the same can be achieved in significantly fewer lines of code by using the higher level methods

In [ ]:
base = tikzinpy.tikzBase()
base += tikzinpy.xaxis(0, 5.9, label = "x", remove_zero = True, offset='-5pt', eps = 0, align = 'right')
base += tikzinpy.yaxis(0, 3.9, label = 'y', remove_zero = True, offset = '-5pt', eps = 0)
#base += pytikz.grid((-2.5,-2.5), (6.5,6.5))

for x,y in zip(data_x, data_y):
    base += point(x,y, shape = 'hollow', color = 'blue')

for x,y in zip(data_x, data_y):
    base += point(x,y + .5, shape = 'hollow', color = 'red')

# also add some lines
for xl,xh, yl, yh in zip(data_x[:-1], data_x[1:], data_y[:-1], data_y[1:]):
    base += line(xl,xh, yl, yh)

# Manual legend for now
# I dont know how to decide the positioning
base += text(4.2,1, 'Blue points', fontsize = 'footnotesize')
base += point(4,1, shape = 'hollow', color = 'blue')

base += text(4.2, 0.7, 'Red points', fontsize = 'footnotesize')
base += point(4, 0.7, shape = 'hollow', color = 'red')


In [ ]:
build(base.figure, 'highlevel')
PDF('highlevel.pdf', size=(700,450))

# Colors
We can also handle matplotlib colormaps

In [ ]:


base = tikzinpy.tikzBase()

base += tikzinpy.xaxis(-2, 6, label = "x", remove_zero = True, offset = '-5pt', align = 'right')
base += tikzinpy.yaxis(-1, 4, label = 'y', remove_zero = True, offset = '-5pt')

base += pointswarm(data_x, data_y, c = data_e, cmap = 'PiYG', shape = 'fill', alpha = 0.6)

In [ ]:
build(base.figure, 'cmap')
PDF('cmap.pdf', size=(700,450))

# Test

In [ ]:
base = tikzinpy.tikzBase()

for x in (-1,1):
    for y in (-1,1):
        base += point(x,y)
base += point(0,0)

base += tikzinpy.xaxis(-2, 6, label = "x", offset = '-5pt', remove_zero = True, align = 'right')
base += tikzinpy.yaxis(0, 4, label = 'cx^2', offset = '-5pt', remove_zero = True)
base += pointswarm(data_x, 0.2*data_y**2, c = data_y, cmap = 'PiYG', shape = 'fill', alpha = 0.6)

base += tikzinpy.colors.colorbar('PiYG', data_y, at_x = -3, at_y=-1, step = 1)

build(base.figure, 'cmap2')
PDF('cmap2.pdf', size=(700,450))

# pgfplots (not even close to working)

In [ ]:
from tikzinpy import pgfplots

In [ ]:
# First we set up some data
x = np.linspace(0,10, 20)
y = x**2

# This sets each point in one of three groups (a,b,c)
# and sets the coloring accordingly
group = np.random.choice(['a','b', 'c'], 20)
style = {'a': 'red', 'b': 'mark=square*, blue', 'c': 'green'}

In [ ]:
base = tikzinpy.tikzBase()
base.add_package('pgfplots')

base += pgfplots.scatterplot(x,y, group, style)
build(base.figure, 'scatter')

In [ ]:
PDF('scatter.pdf', size=(700,450))

In [ ]:
print(base.figure)

In [ ]:
# This is just messing around to try and get colormaps to work
from tikzinpy.colors import build_cmap

In [ ]:
print(build_cmap('magma', data_e).content)